In [1]:
import collections
import time
import item_utils

import dask.bag as db
import requests
import json

from constants import *

In [4]:
def process_json(json_data):
    bin_prices = {}
    auctions = json_data["auctions"]
    for auction in auctions:
        price = auction["starting_bid"]
        if price < auction["highest_bid_amount"]:
            price = auction["highest_bid_amount"]

        if "bin" in auction and not auction["claimed"] and time.time() * 1000 + 60000 < auction["end"]:
            item_id = item_utils.to_custom_item_id(auction)
            if not item_id.startswith("RECOMBOBED"):
                extra = len(auction["extra"][len(auction["item_name"]):].split(" "))
                if item_id not in bin_prices:
                    bin_prices[item_id] = []
                auction_data = {
                    "uuid": auction["uuid"],
                    "price": price,
                    "extra": extra,
                    "raw": auction
                }
                bin_prices[item_id].append(auction_data)

    return bin_prices


In [5]:
total_pages = requests.get("https://api.hypixel.net/skyblock/auctions?key=" + KEY).json()["totalPages"]
urls = []
for page in range(total_pages):
    urls.append("https://api.hypixel.net/skyblock/auctions?key=" + KEY + "&&page=" + str(page))   

    
